#### w/o cot应该只有equation才对

In [22]:
import os, json
import pandas as pd

In [23]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/train_script/data_config/train_gsm8k_direct_no_cot.json",
    "r",
) as f:
    no_cot = json.load(f)

In [24]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/hcot_llm_data.json",
    "r",
) as f:
    no_cot = json.load(f)

In [25]:
no_cot[1]

{'question': 'USER: Question: A car is driving through a tunnel with many turns. After a while, the car must travel through a ring that requires a total of 4 right-hand turns. After the 1st turn, it travels 5 meters. After the 2nd turn, it travels 8 meters. After the 3rd turn, it travels a little further and at the 4th turn, it immediately exits the tunnel. If the car has driven a total of 23 meters around the ring, how far did it have to travel after the 3rd turn?\n Analysis: From the details given, the car has traveled 5 meters at the 1st turn + 8 meters after the 2nd turn + 0 meters after the 4th turn = <<5+8+0=13>>13 meters around the ring.\nIt must therefore have driven 23 total meters – 13 calculated meters = 10 meters after the 3rd turn.\n Answer:10',
 'raw_result': {'id': 'chatcmpl-8SRAPK62ihju3TAq9F26E4ooQdZeq',
  'object': 'chat.completion',
  'created': 1701788305,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '<|im_start

In [35]:
print(no_cot[1]["conversations"][-1]["value"])

The problem involves calculating how far a car traveled after making certain turns in a tunnel. We know that it made 4 right-hand turns, with given distances for the first two turns, no distance covered after the fourth turn, and an overall total distance covered around a ring.

<[COT]>
<equation>Distance_after_1st_and_2nd_turns = Distance_after_1st_turn + Distance_after_2nd_turn = 5 meters + 8 meters = 13 meters</equation>

<[COT]>
<equation>Distance_after_3rd_turn = Total_distance - Distance_after_1st_and_2nd_turns
= 23 meters - 13 meters 
=10 meters</equation>

So, The car must have driven 10 meters after making its third turn.



In [37]:
print(extract_equations(no_cot[1]["conversations"][-1]["value"]))

['Distance_after_1st_and_2nd_turns = Distance_after_1st_turn + Distance_after_2nd_turn = 5 meters + 8 meters = 13 meters', 'Distance_after_3rd_turn = Total_distance - Distance_after_1st_and_2nd_turns\n= 23 meters - 13 meters \n=10 meters']
Distance_after_1st_and_2nd_turns = Distance_after_1st_turn + Distance_after_2nd_turn = 5 meters + 8 meters = 13 meters

Distance_after_3rd_turn = Total_distance - Distance_after_1st_and_2nd_turns
= 23 meters - 13 meters 
=10 meters


In [38]:
import re
def extract_equations(text):
    # 使用 re.DOTALL 使 '.' 匹配任何字符，包括换行符
    equations = re.findall(r"<equation>(.*?)</equation>", text, re.DOTALL)
    # 将找到的所有方程拼接在一起
    combined_equations = "\n\n".join(equations)
    return combined_equations

In [16]:
extract_equations(no_cot[0]["conversations"][-1]['value'])

"Tony's share = 2 * $1750 = $3500\nTotal amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250"

In [39]:
no_cot_new = []
for line in no_cot:
    line_conv = line["conversations"]
    gpt_response = line_conv[1]["value"]
    only_equation = extract_equations(gpt_response)

    line_conv[1]["value"] = only_equation
    no_cot_new.append(line)

In [43]:
import random

random_sample_no_cot_new = random.sample(no_cot_new, 20)
for idx, one_line in enumerate(random_sample_no_cot_new):
    print(one_line["conversations"])
    print(one_line["meta"]["answer"])
    print("Index: {}".format(idx))
    print("*" * 10)

[{'from': 'human', 'value': 'A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?'}, {'from': 'gpt', 'value': '\\text{People on second ship} = 2S\n\n\\text{People on third ship} = 2 * \\text{People on second ship} = 4S\n\nS + (2*S) + (4*S) = Total\\;people\\;consumed\\\\\n7*S = Total\\;people\\;consumed\\\\\n7*S=847\n\nS=\\frac{Total\\;people\\;consumed}{7}\\\\\n=S=\\frac{847}{7}=121'}]
121
Index: 0
**********
[{'from': 'human', 'value': 'Brennan was researching his school project and had to download files from the internet to his computer to use for reference. After downloading 800 files, he deleted 70% of them because they were not helpful. He downloaded 400 more files but again realized that 3/5 of them 

In [46]:
random_sample_no_cot_new[1]

{'question': 'USER: Question: Brennan was researching his school project and had to download files from the internet to his computer to use for reference. After downloading 800 files, he deleted 70% of them because they were not helpful. He downloaded 400 more files but again realized that 3/5 of them were irrelevant. How many valuable files was he left with after deleting the unrelated files he downloaded in the second round?\n Analysis: The number of non-valuable files Brennan downloaded in the first round is 70/100*800 = <<70/100*800=560>>560 files.\nThe number of valuable files Brennan downloaded in the first round is 800-560 = <<800-560=240>>240\nWhen he downloaded 400 new files, there were 3/5*400= <<3/5*400=240>>240 non-useful files, which he deleted again.\nThe total number of valuable files he downloaded in the second round is 400-240 = <<400-240=160>>160\nTo write his research, Brennan had 160+240 = <<160+240=400>>400 useful files to reference to write his research.\n Answer:

#### 不带COT的数据有一些问题，我需要变一下，只保留equation的内容，这次最后要输出答案

#### 带cot的数据什么样子呢

In [8]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/train_script/data_config/train_gsm8k_direct.json", "r") as f:
    with_cot = json.load(f)

In [9]:
with_cot[0]

{'path': '/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/h_cot_direct_conv.json',
 'data_name': 'directly_gsm8k',
 'sample_ratio': 1}

In [10]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/h_cot_direct_conv.json", "r") as f:
    direct_cot = json.load(f)

In [12]:
len(direct_cot)

7430